# Erica Cummings Project 1 DS3002

# Importing Necessary Libraries

In [37]:
import pandas as pd
import io
import os
import pymysql
from sqlalchemy import create_engine


# Declare & Assign Connection Variables for the MySQL Server & Databases with which I'll be Working

In [38]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "ericac"
pwd = "Sch00lus3"

dst_dbname = "university_dw"

# Defining Functions For Creating Dataframes from Local Files 

In [39]:
#this also includes try and except statements as required for the project
def filetoDF(file):
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("Error: File not found.")
    except pd.errors.EmptyDataError:
        print("Error: there is no data in this file.")
    except pd.errors.ParserError:
        print("Error encountered when parsing file.")
    except Exception:
        print("Error: An exception occurred.")
    return df

Creating a Dataframe from file found at https://www.kaggle.com/datasets/padhmam/qs-world-university-rankings-2017-2022?resource=download

In [40]:
uniDF = filetoDF("qs-world-university-rankings-2017-to-2022-V2.csv")
uniDF.head(2)

,university,year,rank_display,score,link,country,city,region,logo,type,research_output,student_faculty_ratio,international_students,size,faculty_count
0,Massachusetts Institute of Technology (MIT),2017,1,100.0,https://www.topuniversities.com/universities/m...,United States,Cambridge,North America,https://www.topuniversities.com/sites/default/...,Private,Very High,4.0,"3,730",M,"3,065"
1,Stanford University,2017,2,98.7,https://www.topuniversities.com/universities/s...,United States,Stanford,North America,https://www.topuniversities.com/sites/default/...,Private,Very High,3.0,"3,879",L,"4,725"


# Dropping Columns

**Requirement 3	Modify the number of columns from the source to the destination, reducing or adding columns"**

I did not think the websites were relevant and were very long compared to the other columns. I also rename rank_display to simply be rank.

In [41]:
drop_cols=['logo','link']
uniDF.drop(drop_cols, axis=1, inplace=True)
uniDF.rename(columns={"rank_display":"rank"}, inplace=True)
uniDF.head()


,university,year,rank,score,country,city,region,type,research_output,student_faculty_ratio,international_students,size,faculty_count
0,Massachusetts Institute of Technology (MIT),2017,1,100.0,United States,Cambridge,North America,Private,Very High,4.0,"3,730",M,"3,065"
1,Stanford University,2017,2,98.7,United States,Stanford,North America,Private,Very High,3.0,"3,879",L,"4,725"
2,Harvard University,2017,3,98.3,United States,Cambridge,North America,Private,Very High,5.0,"5,877",L,"4,646"
3,University of Cambridge,2017,4,97.2,United Kingdom,Cambridge,Europe,Public,Very high,4.0,"7,925",L,"5,800"
4,California Institute of Technology (Caltech),2017,5,96.9,United States,Pasadena,North America,Private,Very High,2.0,692,S,968


# Function for Setting Data Into Databases

In [42]:
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        try:
            df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        except ValueError:
            print("Error: this table already exists")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

# Creating the New Data Warehouse Database

In [43]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

# Load the Transformed DataFrame into the New Data Warehouse by Creating A New Table

**Requirement 4	"The converted (new) file should be written to disk (local file) or written to a SQL database." In this case, a SQL database specifically**

In [44]:
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, uniDF, 'uni_table', db_operation)  

# Summarizing Data

**Requirement 5 "Generate a brief summary of the data file ingestion including: 
a)	Number of records 
b)	Number of columns "**

In [45]:
records= len(uniDF)
print("The number of records is", records)
columns=len(uniDF.columns)
print("The number of columns is", columns)

The number of records is 6482
The number of columns is 13
